# STEP 00 - Import Libraries

In [1]:
import torch
import torch.nn as nn

import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader

from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import time

from timm.data import Mixup
from timm.utils import ModelEmaV3
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
import transformers

from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torchsummary import summary

from convnext_v1 import load_convNext_v1
from convnext_v2 import load_convNext
import math
import warnings
from torch.optim.lr_scheduler import _LRScheduler

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CosineWarmupScheduler(_LRScheduler):
    def __init__(self, optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5, min_lr=1e-6, last_epoch=-1, verbose=False):
        self.num_warmup_steps = num_warmup_steps
        self.num_training_steps = num_training_steps
        self.num_cycles = num_cycles
        self.min_lr = min_lr
        self.base_lrs = [group['lr'] for group in optimizer.param_groups]
        super().__init__(optimizer, last_epoch, verbose)

    def get_lr(self):
        if not self._get_lr_called_within_step:
            warnings.warn("To get the last learning rate computed by the scheduler, "
                          "please use `get_last_lr()`.", UserWarning)
        
        lrs = []
        for base_lr in self.base_lrs:
            if self.last_epoch < self.num_warmup_steps:
                # Linear warmup
                lr = (base_lr - self.min_lr) * self.last_epoch / max(1, self.num_warmup_steps) + self.min_lr
            else:
                # Cosine annealing
                progress = (self.last_epoch - self.num_warmup_steps) / max(1, self.num_training_steps - self.num_warmup_steps)
                lr = self.min_lr + (base_lr - self.min_lr) * 0.5 * (1 + math.cos(math.pi * self.num_cycles * 2.0 * progress))
            lrs.append(lr)
        return lrs

# STEP 01 - Test ConvNeXt V1(sup)

In [3]:
model_v1 = load_convNext_v1()

In [4]:
# 총 파라미터 수 계산
total_params = sum(p.numel() for p in model_v1.parameters())

# 학습 가능한 파라미터 수 계산
trainable_params = sum(p.numel() for p in model_v1.parameters() if p.requires_grad)

print('='*80)
print(f"\nTotal Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}\n")
print('='*80)


Total Parameters: 27,897,028
Trainable Parameters: 27,897,028



In [5]:
# Transforms 정의하기
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.6,1), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=1., scale=(0.02, 0.33)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_dir = '../../data/sports'
batch_size = 512

train_path = data_dir+'/train'
valid_path = data_dir+'/valid'
test_path = data_dir+'/test'

# dataset load
train_data = ImageFolder(train_path, transform=train_transform)
valid_data = ImageFolder(valid_path, transform=test_transform)
test_data = ImageFolder(test_path, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

device = 'cuda:3'
max_norm = 3.0 

model_v1.to(device)

model_ema = None
ema_active = True
if ema_active:
    ema_decay = 0.9998
    model_ema = ModelEmaV3(
        model_v1,
        decay=ema_decay,
    )
    print(f"Using EMA with decay = {ema_decay}")

model_path = ''

mixup = True
if mixup :
    mixup_fn = Mixup(mixup_alpha=.8, 
                    cutmix_alpha=1., 
                    prob=1., 
                    switch_prob=0.5, 
                    mode='batch',
                    label_smoothing=.1,
                    num_classes=100)
    
    criterion = SoftTargetCrossEntropy()
else :
    criterion = LabelSmoothingCrossEntropy(.1)
    
criterion = nn.CrossEntropyLoss(label_smoothing=0.)

epochs = 100

optimizer = optim.AdamW(model_v1.parameters(), lr=4e-3, weight_decay=0.05)
warmup_steps = int(len(train_loader)*(epochs)*0.1)
train_steps = len(train_loader)*(epochs)
scheduler = CosineWarmupScheduler(optimizer, 
                                num_warmup_steps=warmup_steps, 
                                num_training_steps=train_steps,
                                num_cycles=0.5,
                                min_lr=1e-7)

Using EMA with decay = 0.9998


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [6]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

# GradScaler 초기화
scaler = GradScaler()

for i in range(epochs // 100):
    for epoch in range(100):
        model_v1.train()
        start_time = time.time()
        running_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1 + i*100}")
        
        for _, data in pbar:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs, labels = mixup_fn(inputs, labels)
            optimizer.zero_grad()

            # AutoCast 적용
            with autocast():
                outputs = model_v1(inputs)
                loss = criterion(outputs, labels)
                
            # 스케일링된 그라디언트 계산
            scaler.scale(loss).backward()

            # 그라디언트 클리핑 전에 스케일링 제거
            scaler.unscale_(optimizer)
            clip_grad_norm_(model_v1.parameters(), max_norm=max_norm)

            # 옵티마이저 스텝 및 스케일러 업데이트
            scaler.step(optimizer)
            scaler.update()
            
            # EMA 모델 업데이트
            if model_ema is not None:
                model_ema.update(model_v1)
                
            scheduler.step()
                
            lr = optimizer.param_groups[0]["lr"]
            lrs.append(lr)
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)        

        model_v1.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in valid_loader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model_v1(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        val_loss /= len(valid_loader)
        val_losses.append(val_loss)
        
        # 모델 저장
        total_loss = val_loss + epoch_loss
        if total_loss < best_loss:
            best_loss = total_loss
            model_save = False
            if model_save:
                torch.save(model_v1.state_dict(), model_path)

        epoch_duration = time.time() - start_time
        training_time += epoch_duration
        
        text = f'\tLoss: {epoch_loss:.4f}, Val_Loss: {val_loss:.4f}, Total Mean Loss: {total_loss/2:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
        
        if model_save:
            text += f' - model saved!'
            model_save = False

        print(text)

    # 예측 수행 및 레이블 저장
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_v1(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 혼동 행렬 생성
    cm = confusion_matrix(all_labels, all_preds)

    # 예측과 실제 레이블
    y_true = all_labels  # 실제 레이블
    y_pred = all_preds  # 모델에 의해 예측된 레이블

    # 전체 데이터셋에 대한 정확도
    accuracy = accuracy_score(y_true, y_pred)

    # 평균 정밀도, 리콜, F1-Score ('weighted')
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # 판다스 데이터프레임으로 결과 정리
    performance_metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Value': [accuracy, precision, recall, f1_score]
    })

    # 데이터프레임 출력
    print(f"\n[{i*100+100} epoch result]\n", performance_metrics)

Epoch 1: 100%|██████████| 27/27 [00:56<00:00,  2.08s/it]


	Loss: 4.4774, Val_Loss: 4.0328, Total Mean Loss: 4.2551, LR: 0.00040008999999999997, Duration: 57.10 sec


Epoch 2: 100%|██████████| 27/27 [00:55<00:00,  2.05s/it]


	Loss: 4.3636, Val_Loss: 3.9938, Total Mean Loss: 4.1787, LR: 0.0008000799999999999, Duration: 56.24 sec


Epoch 3: 100%|██████████| 27/27 [00:55<00:00,  2.06s/it]


...

	Loss: 2.1196, Val_Loss: 0.7607, Total Mean Loss: 1.4401, LR: 4.971777682864596e-06, Duration: 56.02 sec


Epoch 99: 100%|██████████| 27/27 [00:55<00:00,  2.04s/it]


	Loss: 2.0866, Val_Loss: 0.7598, Total Mean Loss: 1.4232, LR: 1.3183155031594304e-06, Duration: 56.12 sec


Epoch 100: 100%|██████████| 27/27 [00:55<00:00,  2.04s/it]


	Loss: 2.1089, Val_Loss: 0.7597, Total Mean Loss: 1.4343, LR: 1e-07, Duration: 56.03 sec

[100 epoch result]
       Metric     Value
0   Accuracy  0.876000
1  Precision  0.896143
2     Recall  0.876000
3   F1 Score  0.872440


In [7]:
model_v1.cpu()
del model_v1

# STEP 02 - Test ConvNeXt V2(sup)

In [8]:
model_v2 = load_convNext()

# 총 파라미터 수 계산
total_params = sum(p.numel() for p in model_v2.parameters())

# 학습 가능한 파라미터 수 계산
trainable_params = sum(p.numel() for p in model_v2.parameters() if p.requires_grad)

print('='*80)
print(f"\nTotal Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}\n")
print('='*80)


Total Parameters: 27,943,396
Trainable Parameters: 27,943,396



In [9]:
model_v2.to(device)

model_ema = None
ema_active = True
if ema_active:
    ema_decay = 0.9998
    model_ema = ModelEmaV3(
        model_v2,
        decay=ema_decay,
    )
    print(f"Using EMA with decay = {ema_decay}")

model_path = ''

mixup = True
if mixup :
    mixup_fn = Mixup(mixup_alpha=.8, 
                    cutmix_alpha=1., 
                    prob=1., 
                    switch_prob=0.5, 
                    mode='batch',
                    label_smoothing=.1,
                    num_classes=100)
    
    criterion = SoftTargetCrossEntropy()
else :
    criterion = LabelSmoothingCrossEntropy(.1)
    
criterion = nn.CrossEntropyLoss(label_smoothing=0.)

epochs = 100

optimizer = optim.AdamW(model_v2.parameters(), lr=4e-3, weight_decay=0.05)
warmup_steps = int(len(train_loader)*(epochs)*0.1)
train_steps = len(train_loader)*(epochs)
scheduler = CosineWarmupScheduler(optimizer, 
                                num_warmup_steps=warmup_steps, 
                                num_training_steps=train_steps,
                                num_cycles=0.5,
                                min_lr=1e-7)

Using EMA with decay = 0.9998


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [10]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

# GradScaler 초기화
scaler = GradScaler()

for i in range(epochs // 100):
    for epoch in range(100):
        model_v2.train()
        start_time = time.time()
        running_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1 + i*100}")
        
        for _, data in pbar:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs, labels = mixup_fn(inputs, labels)
            optimizer.zero_grad()

            # AutoCast 적용
            with autocast():
                outputs = model_v2(inputs)
                loss = criterion(outputs, labels)
                
            # 스케일링된 그라디언트 계산
            scaler.scale(loss).backward()

            # 그라디언트 클리핑 전에 스케일링 제거
            scaler.unscale_(optimizer)
            clip_grad_norm_(model_v2.parameters(), max_norm=max_norm)

            # 옵티마이저 스텝 및 스케일러 업데이트
            scaler.step(optimizer)
            scaler.update()
            
            # EMA 모델 업데이트
            if model_ema is not None:
                model_ema.update(model_v2)
                
            scheduler.step()
                
            lr = optimizer.param_groups[0]["lr"]
            lrs.append(lr)
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)        

        model_v2.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in valid_loader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model_v2(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        val_loss /= len(valid_loader)
        val_losses.append(val_loss)
        
        # 모델 저장
        total_loss = val_loss + epoch_loss
        if total_loss < best_loss:
            best_loss = total_loss
            model_save = False
            if model_save:
                torch.save(model_v2.state_dict(), model_path)

        epoch_duration = time.time() - start_time
        training_time += epoch_duration
        
        text = f'\tLoss: {epoch_loss:.4f}, Val_Loss: {val_loss:.4f}, Total Mean Loss: {total_loss/2:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
        
        if model_save:
            text += f' - model saved!'
            model_save = False

        print(text)

    # 예측 수행 및 레이블 저장
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_v2(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 혼동 행렬 생성
    cm = confusion_matrix(all_labels, all_preds)

    # 예측과 실제 레이블
    y_true = all_labels  # 실제 레이블
    y_pred = all_preds  # 모델에 의해 예측된 레이블

    # 전체 데이터셋에 대한 정확도
    accuracy = accuracy_score(y_true, y_pred)

    # 평균 정밀도, 리콜, F1-Score ('weighted')
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # 판다스 데이터프레임으로 결과 정리
    performance_metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Value': [accuracy, precision, recall, f1_score]
    })

    # 데이터프레임 출력
    print(f"\n[{i*100+100} epoch result]\n", performance_metrics)

Epoch 1: 100%|██████████| 27/27 [01:06<00:00,  2.46s/it]


	Loss: 4.4953, Val_Loss: 3.9957, Total Mean Loss: 4.2455, LR: 0.00040008999999999997, Duration: 67.57 sec


Epoch 2: 100%|██████████| 27/27 [01:06<00:00,  2.46s/it]


	Loss: 4.3383, Val_Loss: 3.8066, Total Mean Loss: 4.0724, LR: 0.0008000799999999999, Duration: 67.57 sec


Epoch 3: 100%|██████████| 27/27 [01:06<00:00,  2.47s/it]


...

	Loss: 2.1632, Val_Loss: 0.7543, Total Mean Loss: 1.4588, LR: 4.971777682864596e-06, Duration: 67.64 sec


Epoch 99: 100%|██████████| 27/27 [01:06<00:00,  2.46s/it]


	Loss: 2.1685, Val_Loss: 0.7549, Total Mean Loss: 1.4617, LR: 1.3183155031594304e-06, Duration: 67.57 sec


Epoch 100: 100%|██████████| 27/27 [01:06<00:00,  2.46s/it]


	Loss: 2.0983, Val_Loss: 0.7549, Total Mean Loss: 1.4266, LR: 1e-07, Duration: 67.38 sec

[100 epoch result]
       Metric     Value
0   Accuracy  0.894000
1  Precision  0.912238
2     Recall  0.894000
3   F1 Score  0.891306


# STEP 03 - Test ConvNeXt V2 with FCMAE

## 00.MASK

In [54]:
import torch

# Define the upsample_mask and a simple example
def upsample_mask(mask, scale):
    assert len(mask.shape) == 2
    p = int(mask.shape[1] ** .5)
    return mask.reshape(-1, p, p).repeat_interleave(scale, axis=1).repeat_interleave(scale, axis=2)

# 예시 마스크와 데이터
mask = torch.tensor([[0, 1, 0, 1],
                     [1, 0, 1, 0]])
x = torch.arange(1., 17.).reshape(1, 1, 4, 4)  # 4x4 데이터, 채널과 배치 차원 포함

# 마스크 업샘플링
upsampled_mask = upsample_mask(mask, 2)  # 스케일을 2로 설정
upsampled_mask = upsampled_mask.unsqueeze(1)  # 채널 차원 추가, 올바른 변수 이름 사용

# 데이터에 마스크 적용
x_orig = x.clone()  # 원본 데이터 복사
x_masked = x * (1. - upsampled_mask.type_as(x))  # 마스크 적용, 타입 일치 시키기

print(f"\n원본 데이터\n{x_orig}")
print(f"\n마스크\n{upsampled_mask}")
print(f"\n마스크 된 데이터\n{x_masked}")


원본 데이터
tensor([[[[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

마스크
tensor([[[[0, 0, 1, 1],
          [0, 0, 1, 1],
          [0, 0, 1, 1],
          [0, 0, 1, 1]]],


        [[[1, 1, 0, 0],
          [1, 1, 0, 0],
          [1, 1, 0, 0],
          [1, 1, 0, 0]]]])

마스크 된 데이터
tensor([[[[ 1.,  2.,  0.,  0.],
          [ 5.,  6.,  0.,  0.],
          [ 9., 10.,  0.,  0.],
          [13., 14.,  0.,  0.]]],


        [[[ 0.,  0.,  3.,  4.],
          [ 0.,  0.,  7.,  8.],
          [ 0.,  0., 11., 12.],
          [ 0.,  0., 15., 16.]]]])


## 01.TRAIN FCMAE

In [3]:
from model.fcmae import convnextv2_fcmae_tiny

model = convnextv2_fcmae_tiny()

/opt/conda/lib/python3.10/site-packages/MinkowskiEngine-0.5.4-py3.10-linux-x86_64.egg/MinkowskiEngine/__init__.py:36: UserWarning: The environment variable `OMP_NUM_THREADS` not set. MinkowskiEngine will automatically set `OMP_NUM_THREADS=16`. If you want to set `OMP_NUM_THREADS` manually, please export it on the command line before running a python script. e.g. `export OMP_NUM_THREADS=12; python your_program.py`. It is recommended to set it below 24.
  warnings.warn(


In [4]:
# Transforms 정의하기
train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.2, 1.0), interpolation=3),  # 3 is bicubic
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

data_dir = '../../data/sports/'
batch_size = 800
train_path = data_dir

# dataset load
train_data = ImageFolder(train_path, transform=train_transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)

len(train_loader)
device = 'cuda:0'
model.to(device)

model_path = '../../model/convnext/fcmae.pt'

epochs = 500
optimizer = optim.AdamW(model.parameters(), lr=1.5e-4, weight_decay=0.05, betas=(0.9, 0.95))

warmup_steps = int(len(train_loader)*(epochs)*0.1)
train_steps = len(train_loader)*(epochs)
scheduler = CosineWarmupScheduler(optimizer, 
                                num_warmup_steps=50, 
                                num_training_steps=500,
                                num_cycles=0.5,
                                min_lr=1e-7)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [ ]:

training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

for epoch in range(epochs):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        optimizer.zero_grad()
        
        samples= data[0].to(device)
        loss, _, _ = model(samples, mask_ratio=0.6)
        
        loss.backward()
        optimizer.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        
    
    # 모델 저장 로직 조정
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        if epoch > (epochs // 2) :
            torch.save(model.state_dict(), model_path)
            model_saved_text = ' - model saved!'
        else :
            model_saved_text = ' - model save pass'
    else:
        model_saved_text = ''
    
    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss:,.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec{model_saved_text}'
    print(text)

    # 에폭마다 스케줄러 업데이트
    scheduler.step()

## 02.SUP after FCMAE

### Load Model & pretrained Weights

In [3]:
from model.convnextv2 import load_convNext

model = load_convNext(droppath=0.2)

In [4]:
model.state_dict().keys()

odict_keys(['stem.stem_conv.weight', 'stem.stem_conv.bias', 'stem.stem_ln.weight', 'stem.stem_ln.bias', 'downsample_layers.0.stem_conv.weight', 'downsample_layers.0.stem_conv.bias', 'downsample_layers.0.stem_ln.weight', 'downsample_layers.0.stem_ln.bias', 'downsample_layers.1.ds_ln0.weight', 'downsample_layers.1.ds_ln0.bias', 'downsample_layers.1.ds_conv1.weight', 'downsample_layers.1.ds_conv1.bias', 'downsample_layers.2.ds_ln1.weight', 'downsample_layers.2.ds_ln1.bias', 'downsample_layers.2.ds_conv2.weight', 'downsample_layers.2.ds_conv2.bias', 'downsample_layers.3.ds_ln2.weight', 'downsample_layers.3.ds_ln2.bias', 'downsample_layers.3.ds_conv3.weight', 'downsample_layers.3.ds_conv3.bias', 'stages.0.0.dwconv.weight', 'stages.0.0.dwconv.bias', 'stages.0.0.layernorm.weight', 'stages.0.0.layernorm.bias', 'stages.0.0.pwconv1.weight', 'stages.0.0.pwconv1.bias', 'stages.0.0.grn.gamma', 'stages.0.0.grn.beta', 'stages.0.0.pwconv2.weight', 'stages.0.0.pwconv2.bias', 'stages.0.1.dwconv.weight',

In [5]:
model.state_dict()['downsample_layers.0.stem_conv.weight'][0]

tensor([[[ 0.0629,  0.0952,  0.0748,  0.0902],
         [ 0.1215,  0.0440,  0.0270,  0.1279],
         [-0.0605, -0.0492,  0.0668,  0.0991],
         [-0.1112, -0.0794, -0.0212, -0.1278]],

        [[ 0.1158,  0.1106, -0.0207,  0.1087],
         [-0.1291, -0.1180,  0.0860, -0.1328],
         [-0.0183,  0.0993,  0.0566,  0.0913],
         [-0.1206,  0.1105, -0.0288,  0.0171]],

        [[-0.0108, -0.0132,  0.1078, -0.0028],
         [ 0.1139,  0.0600,  0.0073, -0.0734],
         [-0.1233, -0.0571, -0.0904,  0.1424],
         [ 0.1076,  0.1441, -0.0413, -0.1250]]])

In [6]:
model.state_dict()['stages.0.0.dwconv.weight'][0]

tensor([[[ 0.0157, -0.1240, -0.0387,  0.1137, -0.0658,  0.0407,  0.0408],
         [-0.0975, -0.0339,  0.0239,  0.0722, -0.0470, -0.0437, -0.0265],
         [ 0.1292,  0.0394,  0.0845,  0.1158, -0.0432,  0.0255,  0.0569],
         [-0.0611, -0.0005,  0.1062,  0.0153,  0.0700, -0.1000,  0.1077],
         [ 0.1302,  0.1213, -0.1228,  0.1067,  0.1099,  0.0903,  0.0400],
         [-0.0856,  0.1377,  0.1230,  0.0098, -0.0291, -0.0246,  0.1001],
         [-0.0116,  0.0964,  0.0121,  0.0938, -0.1285, -0.0569, -0.0039]]])

### Remap Weights

In [7]:
from collections import OrderedDict

def remap_checkpoint_keys(ckpt):
    new_ckpt = OrderedDict()
    for k, v in ckpt.items():
        if k.startswith('encoder'):
            k = '.'.join(k.split('.')[1:]) # remove encoder in the name
        if k.endswith('kernel'):
            k = '.'.join(k.split('.')[:-1]) # remove kernel in the name
            new_k = k + '.weight'
            if len(v.shape) == 3: # resahpe standard convolution
                kv, in_dim, out_dim = v.shape
                ks = int(math.sqrt(kv))
                new_ckpt[new_k] = v.permute(2, 1, 0).\
                    reshape(out_dim, in_dim, ks, ks).transpose(3, 2)
            elif len(v.shape) == 2: # reshape depthwise convolution
                kv, dim = v.shape
                ks = int(math.sqrt(kv))
                new_ckpt[new_k] = v.permute(1, 0).\
                    reshape(dim, 1, ks, ks).transpose(3, 2)
            continue
        elif 'ln' in k or 'linear' in k:
            k = k.split('.')
            k.pop(-2) # remove ln and linear in the name
            new_k = '.'.join(k)
        else:
            new_k = k
        new_ckpt[new_k] = v

    # reshape grn affine parameters and biases
    for k, v in new_ckpt.items():
        if k.endswith('bias') and len(v.shape) != 1:
            new_ckpt[k] = v.reshape(-1)
        elif 'grn' in k:
            new_ckpt[k] = v.unsqueeze(0).unsqueeze(1)
    return new_ckpt

def load_state_dict(model, state_dict, prefix='', ignore_missing="relative_position_index"):
    missing_keys = []
    unexpected_keys = []
    error_msgs = []
    # copy state_dict so _load_from_state_dict can modify it
    metadata = getattr(state_dict, '_metadata', None)
    state_dict = state_dict.copy()
    if metadata is not None:
        state_dict._metadata = metadata

    def load(module, prefix=''):
        local_metadata = {} if metadata is None else metadata.get(
            prefix[:-1], {})
        module._load_from_state_dict(
            state_dict, prefix, local_metadata, True, missing_keys, unexpected_keys, error_msgs)
        for name, child in module._modules.items():
            if child is not None:
                load(child, prefix + name + '.')

    load(model, prefix=prefix)

    warn_missing_keys = []
    ignore_missing_keys = []
    for key in missing_keys:
        keep_flag = True
        for ignore_key in ignore_missing.split('|'):
            if ignore_key in key:
                keep_flag = False
                break
        if keep_flag:
            warn_missing_keys.append(key)
        else:
            ignore_missing_keys.append(key)

    missing_keys = warn_missing_keys

    if len(missing_keys) > 0:
        print("Weights of {} not initialized from pretrained model: {}".format(
            model.__class__.__name__, missing_keys))
    if len(unexpected_keys) > 0:
        print("Weights from pretrained model not used in {}: {}".format(
            model.__class__.__name__, unexpected_keys))
    if len(ignore_missing_keys) > 0:
        print("Ignored weights of {} not initialized from pretrained model: {}".format(
            model.__class__.__name__, ignore_missing_keys))
    if len(error_msgs) > 0:
        print('\n'.join(error_msgs))

In [8]:
pretrain_path = '../../model/convnext/fcmae.pt'
checkpoint_model = torch.load(pretrain_path, map_location='cpu')

state_dict = model.state_dict()
for k in ['head.weight', 'head.bias']:
    if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
        print(f"Removing key {k} from head of pretrained checkpoint")
        del checkpoint_model[k]

# remove decoder weights
checkpoint_model_keys = list(checkpoint_model.keys())
for k in checkpoint_model_keys:
    if 'decoder' in k or 'mask_token'in k or \
        'proj' in k or 'pred' in k:
        print(f"Removing key {k} from decoder of pretrained checkpoint")
        del checkpoint_model[k]

checkpoint_model = remap_checkpoint_keys(checkpoint_model)
load_state_dict(model, checkpoint_model, prefix='')

Removing key mask_token from decoder of pretrained checkpoint
Removing key proj.weight from decoder of pretrained checkpoint
Removing key proj.bias from decoder of pretrained checkpoint
Removing key decoder.0.dwconv.weight from decoder of pretrained checkpoint
Removing key decoder.0.dwconv.bias from decoder of pretrained checkpoint
Removing key decoder.0.layernorm.weight from decoder of pretrained checkpoint
Removing key decoder.0.layernorm.bias from decoder of pretrained checkpoint
Removing key decoder.0.pwconv1.weight from decoder of pretrained checkpoint
Removing key decoder.0.pwconv1.bias from decoder of pretrained checkpoint
Removing key decoder.0.grn.gamma from decoder of pretrained checkpoint
Removing key decoder.0.grn.beta from decoder of pretrained checkpoint
Removing key decoder.0.pwconv2.weight from decoder of pretrained checkpoint
Removing key decoder.0.pwconv2.bias from decoder of pretrained checkpoint
Removing key pred.weight from decoder of pretrained checkpoint
Removing

In [9]:
# manually initialize fc layer
nn.init.trunc_normal_(model.fc.weight, std=2e-5)
torch.nn.init.constant_(model.fc.bias, 0.)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.], requires_grad=True)

In [10]:
model.state_dict()['downsample_layers.0.stem_conv.weight'][0]

tensor([[[ 0.2945,  0.7202, -0.3333, -0.2289],
         [-0.5944,  0.4452,  0.9049, -0.2540],
         [ 0.4562, -0.8116,  1.4202,  0.4324],
         [ 0.1777,  0.3165, -1.3547,  0.2427]],

        [[-1.5330, -1.2280, -0.4023, -0.5692],
         [-0.5322, -0.0219,  0.3301, -1.0079],
         [ 0.4587, -0.0214, -0.6747, -0.7620],
         [ 0.1962, -0.4949, -0.0375, -0.0354]],

        [[-0.8695,  0.9264, -0.2321, -1.5825],
         [-0.4034, -0.4807,  0.2547, -0.3684],
         [-0.6111, -1.2094,  0.8702,  0.9287],
         [ 1.1989,  0.0221, -0.5370,  0.5360]]])

In [11]:
model.state_dict()['downsample_layers.0.stem_ln.weight']

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.])

In [12]:
model.state_dict()['stages.0.0.dwconv.weight'][0]

tensor([[[ 1.2615, -0.2950, -0.1866,  0.1731,  0.0968, -0.2890, -0.9205],
         [-0.5179,  0.9794, -0.8361,  0.5204, -0.4569,  0.2246, -0.1680],
         [-0.5284, -0.1686, -0.2800,  1.1796, -1.3214,  0.7117, -1.5333],
         [-0.2485, -0.1683,  1.0382,  0.2789,  0.3089,  0.2259, -0.1799],
         [ 1.6295,  0.7848,  0.8198, -1.2768, -0.3535, -1.3434,  0.1347],
         [-0.9390,  0.4803,  0.1530,  0.2759, -0.5772, -0.9058,  0.6663],
         [ 0.7074,  0.3372, -0.4517, -0.3804,  0.4552,  0.9987,  0.3588]]])

In [13]:
# 총 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())

# 학습 가능한 파라미터 수 계산
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print('='*80)
print(f"\nTotal Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}\n")
print('='*80)


Total Parameters: 27,943,396
Trainable Parameters: 27,943,396



In [14]:
# Transforms 정의하기
train_transform = transforms.Compose([
    transforms.TrivialAugmentWide(interpolation=F.InterpolationMode.BICUBIC),
    transforms.RandomResizedCrop(224, scale=(0.6,1), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_dir = '../../data/sports'
batch_size = 320

train_path = data_dir+'/train'
valid_path = data_dir+'/valid'
test_path = data_dir+'/test'

# dataset load
train_data = ImageFolder(train_path, transform=train_transform)
valid_data = ImageFolder(valid_path, transform=test_transform)
test_data = ImageFolder(test_path, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [15]:
device = 'cuda:4'
max_norm = 3.0 

model.to(device)

model_ema = None
ema_active = True
if ema_active:
    ema_decay = 0.9998
    model_ema = ModelEmaV3(
        model,
        decay=ema_decay,
    )
    print(f"Using EMA with decay = {ema_decay}")

model_path = ''

mixup = True
if mixup :
    mixup_fn = Mixup(mixup_alpha=.8, 
                    cutmix_alpha=1., 
                    prob=1., 
                    switch_prob=0.5, 
                    mode='batch',
                    label_smoothing=.1,
                    num_classes=100)
    
    criterion = SoftTargetCrossEntropy()
else :
    criterion = LabelSmoothingCrossEntropy(.1)
    
criterion = nn.CrossEntropyLoss(label_smoothing=0.)

Using EMA with decay = 0.9998


### Without LayerWise Learning Rate Decay

In [16]:
epochs = 100

optimizer = optim.AdamW(model.parameters(), lr=4e-3, weight_decay=0.05)
warmup_steps = int(len(train_loader)*(epochs)*0.1)
train_steps = len(train_loader)*(epochs)
scheduler = CosineWarmupScheduler(optimizer, 
                                num_warmup_steps=warmup_steps, 
                                num_training_steps=train_steps,
                                num_cycles=0.5,
                                min_lr=1e-7)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [17]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')
model_save = False
torch.backends.cudnn.benchmark = True

for i in range(epochs // 100):
    for epoch in range(100):
        model.train()
        start_time = time.time()
        running_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1 + i*100}")
        
        for _, data in pbar:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs, labels = mixup_fn(inputs, labels)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
                
            loss.backward()
            # 그라디언트 클리핑 적용
            clip_grad_norm_(model.parameters(), max_norm=max_norm)
            optimizer.step()
            
            # EMA 모델 업데이트, 필요한 경우
            if model_ema is not None:
                model_ema.update(model)
                
            scheduler.step()
                
            lr = optimizer.param_groups[0]["lr"]
            lrs.append(lr)
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in valid_loader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        val_loss /= len(valid_loader)
        val_losses.append(val_loss)
        
        # 모델 저장 조건 수정
        total_loss = val_loss + epoch_loss
        if total_loss < best_loss:
            best_loss = total_loss
            # torch.save(model.state_dict(), model_path)
            model_save = True
            save_text = ' - model saved!'
        else:
            save_text = ''

        epoch_duration = time.time() - start_time
        training_time += epoch_duration
        
        text = f'\tLoss: {epoch_loss:.4f}, Val_Loss: {val_loss:.4f}, Total Mean Loss: {total_loss/2:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec{save_text}'
        print(text)

    # 예측 수행 및 레이블 저장
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 혼동 행렬 생성
    cm = confusion_matrix(all_labels, all_preds)

    # 예측과 실제 레이블
    y_true = all_labels  # 실제 레이블
    y_pred = all_preds  # 모델에 의해 예측된 레이블

    # 전체 데이터셋에 대한 정확도
    accuracy = accuracy_score(y_true, y_pred)

    # 평균 정밀도, 리콜, F1-Score ('weighted')
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # 판다스 데이터프레임으로 결과 정리
    performance_metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Value': [accuracy, precision, recall, f1_score]
    })

    # 데이터프레임 출력
    print(f"\n[{i*100+100} epoch result]\n", performance_metrics)

Epoch 1:   0%|          | 0/43 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 43/43 [01:41<00:00,  2.35s/it]


	Loss: 4.5177, Val_Loss: 4.1077, Total Mean Loss: 4.3127, LR: 0.00040009, Duration: 102.63 sec - model saved!


Epoch 2: 100%|██████████| 43/43 [01:39<00:00,  2.31s/it]


	Loss: 4.4140, Val_Loss: 4.0270, Total Mean Loss: 4.2205, LR: 0.00080008, Duration: 100.71 sec - model saved!


...

	Loss: 4.2681, Val_Loss: 3.6038, Total Mean Loss: 3.9359, LR: 0.0020000499999999997, Duration: 102.34 sec - model saved!


Epoch 6: 100%|██████████| 43/43 [01:41<00:00,  2.36s/it]


	Loss: 4.2788, Val_Loss: 3.5822, Total Mean Loss: 3.9305, LR: 0.0024000399999999996, Duration: 102.91 sec - model saved!


Epoch 7:  23%|██▎       | 10/43 [00:25<01:23,  2.53s/it]


KeyboardInterrupt: 

### LayerWise Learning Decay

In [15]:
def LLRD_ConvNeXt(depths=[3,3,9,3]):
    stage = 0
    layer_names = []
    for depth in depths:
        if stage == 0:
            layer_names.append(f'downsampling_layers.{stage}.stem_conv.weight')
            layer_names.append(f'downsampling_layers.{stage}.stem_conv.bias')
            layer_names.append(f'downsampling_layers.{stage}.stem_ln.weight')
            layer_names.append(f'downsampling_layers.{stage}.stem_ln.bias')
        else :
            layer_names.append(f'downsampling_layers.{stage}.ds_ln.weight')
            layer_names.append(f'downsampling_layers.{stage}.ds_ln.bias')
            layer_names.append(f'downsampling_layers.{stage}.ds_conv.weight')
            layer_names.append(f'downsampling_layers.{stage}.ds_conv.bias')
        
        for i in range(depth):
            layer_names.append(f'stages.{stage}.{i}.dwconv.weight')
            layer_names.append(f'stages.{stage}.{i}.dwconv.bias')
            layer_names.append(f'stages.{stage}.{i}.layernorm.weight')
            layer_names.append(f'stages.{stage}.{i}.layernorm.bias')
            layer_names.append(f'stages.{stage}.{i}.pwconv1.weight')
            layer_names.append(f'stages.{stage}.{i}.pwconv1.bias')
            layer_names.append(f'stages.{stage}.{i}.grn.gamma')
            layer_names.append(f'stages.{stage}.{i}.grn.beta')            
            layer_names.append(f'stages.{stage}.{i}.pwconv2.weight')
            layer_names.append(f'stages.{stage}.{i}.pwconv2.bias')
        stage += 1
    
    layer_names.append('layernorm.weight')
    layer_names.append('layernorm.bias')
    layer_names.append('fc.weight')
    layer_names.append('fc.bias')
    return layer_names

In [17]:
# LLRD
layer_names = []
for i, (name, params) in enumerate(model.named_parameters()):
    layer_names.append(name)
    
layer_names.reverse()

lr      = 8e-3  
lr_mult = 0.9  
weight_decay = 0.05 

param_groups = []
prev_group_name = layer_names[0].split('.')[0]

for idx, name in enumerate(layer_names):
    
    cur_group_name = name.split('.')[0]
    
    if cur_group_name != prev_group_name:
        lr *= lr_mult
    prev_group_name = cur_group_name
    
    print(f"{idx}: {name}'s lr={lr}")
    
    param_groups += [{'params': [ p for n, p in model.named_parameters() if n == name and p.requires_grad],
                      'lr' : lr,
                      'weight_decay': weight_decay}]

0: fc.bias's lr=0.008
1: fc.weight's lr=0.008
2: layernorm.bias's lr=0.007200000000000001
3: layernorm.weight's lr=0.007200000000000001
4: stages.3.2.pwconv2.bias's lr=0.0064800000000000005
5: stages.3.2.pwconv2.weight's lr=0.0064800000000000005
6: stages.3.2.grn.beta's lr=0.0064800000000000005
7: stages.3.2.grn.gamma's lr=0.0064800000000000005
8: stages.3.2.pwconv1.bias's lr=0.0064800000000000005
9: stages.3.2.pwconv1.weight's lr=0.0064800000000000005
10: stages.3.2.layernorm.bias's lr=0.0064800000000000005
11: stages.3.2.layernorm.weight's lr=0.0064800000000000005
12: stages.3.2.dwconv.bias's lr=0.0064800000000000005
13: stages.3.2.dwconv.weight's lr=0.0064800000000000005
14: stages.3.1.pwconv2.bias's lr=0.0064800000000000005
15: stages.3.1.pwconv2.weight's lr=0.0064800000000000005
16: stages.3.1.grn.beta's lr=0.0064800000000000005
17: stages.3.1.grn.gamma's lr=0.0064800000000000005
18: stages.3.1.pwconv1.bias's lr=0.0064800000000000005
19: stages.3.1.pwconv1.weight's lr=0.0064800000

In [18]:
# LLRD
layer_names = LLRD_ConvNeXt()
    
layer_names.reverse()

lr0     = 8e-3  
lr_mult = 0.9  
weight_decay = 0.05 

param_groups = []
prev_group_name = layer_names[0].split('.')[0]

lr = lr0
for idx, name in enumerate(layer_names):
    
    cur_group_name = name.split('.')[0] 
    
    if cur_group_name != prev_group_name:
        if ("ln" in name) | ("layernorm" in name) | ('grn' in name):
            pass
        else:
            lr *= lr_mult
    prev_group_name = cur_group_name
    
    if "bias" in name :
        print(f"{idx}: {name} | lr={lr0}")
        param_groups += [{'params': [ p for n, p in model.named_parameters() if n == name and p.requires_grad],
                        'lr' : lr0}]
    elif ("ln" in name) | ("layernorm" in name) | ('grn' in name):
        param_groups += [{'params': [ p for n, p in model.named_parameters() if n == name and p.requires_grad],
                        'lr' : lr0}]   
        print(f"{idx}: {name} | lr={lr0}")
    else :        
        param_groups += [{'params': [ p for n, p in model.named_parameters() if n == name and p.requires_grad],
                        'lr' : lr,
                        'weight_decay': weight_decay}]
        print(f"{idx}: {name} | lr={lr}, weight decay={weight_decay}")

0: fc.bias | lr=0.008
1: fc.weight | lr=0.008, weight decay=0.05
2: layernorm.bias | lr=0.008
3: layernorm.weight | lr=0.008
4: stages.3.2.pwconv2.bias | lr=0.008
5: stages.3.2.pwconv2.weight | lr=0.007200000000000001, weight decay=0.05
6: stages.3.2.grn.beta | lr=0.008
7: stages.3.2.grn.gamma | lr=0.008
8: stages.3.2.pwconv1.bias | lr=0.008
9: stages.3.2.pwconv1.weight | lr=0.007200000000000001, weight decay=0.05
10: stages.3.2.layernorm.bias | lr=0.008
11: stages.3.2.layernorm.weight | lr=0.008
12: stages.3.2.dwconv.bias | lr=0.008
13: stages.3.2.dwconv.weight | lr=0.007200000000000001, weight decay=0.05
14: stages.3.1.pwconv2.bias | lr=0.008
15: stages.3.1.pwconv2.weight | lr=0.007200000000000001, weight decay=0.05
16: stages.3.1.grn.beta | lr=0.008
17: stages.3.1.grn.gamma | lr=0.008
18: stages.3.1.pwconv1.bias | lr=0.008
19: stages.3.1.pwconv1.weight | lr=0.007200000000000001, weight decay=0.05
20: stages.3.1.layernorm.bias | lr=0.008
21: stages.3.1.layernorm.weight | lr=0.008
22:

In [19]:
epochs = 100

optimizer = optim.AdamW(param_groups, betas=(0.9,0.999))
warmup_steps = int(len(train_loader)*(epochs)*0.1)
train_steps = len(train_loader)*(epochs)
scheduler = CosineWarmupScheduler(optimizer, 
                                num_warmup_steps=warmup_steps, 
                                num_training_steps=train_steps,
                                num_cycles=0.5,
                                min_lr=1e-7)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [20]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')
model_save = False
torch.backends.cudnn.benchmark = True

for i in range(epochs // 100):
    for epoch in range(100):
        model.train()
        start_time = time.time()
        running_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1 + i*100}")
        
        for _, data in pbar:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs, labels = mixup_fn(inputs, labels)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
                
            loss.backward()
            # 그라디언트 클리핑 적용
            clip_grad_norm_(model.parameters(), max_norm=max_norm)
            optimizer.step()
            
            # EMA 모델 업데이트, 필요한 경우
            if model_ema is not None:
                model_ema.update(model)
                
            scheduler.step()
                
            lr = optimizer.param_groups[0]["lr"]
            lrs.append(lr)
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in valid_loader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        val_loss /= len(valid_loader)
        val_losses.append(val_loss)
        
        # 모델 저장 조건 수정
        total_loss = val_loss + epoch_loss
        if total_loss < best_loss:
            best_loss = total_loss
            # torch.save(model.state_dict(), model_path)
            model_save = True
            save_text = ' - model saved!'
        else:
            save_text = ''

        epoch_duration = time.time() - start_time
        training_time += epoch_duration
        
        text = f'\tLoss: {epoch_loss:.4f}, Val_Loss: {val_loss:.4f}, Total Mean Loss: {total_loss/2:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec{save_text}'
        print(text)

    # 예측 수행 및 레이블 저장
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 혼동 행렬 생성
    cm = confusion_matrix(all_labels, all_preds)

    # 예측과 실제 레이블
    y_true = all_labels  # 실제 레이블
    y_pred = all_preds  # 모델에 의해 예측된 레이블

    # 전체 데이터셋에 대한 정확도
    accuracy = accuracy_score(y_true, y_pred)

    # 평균 정밀도, 리콜, F1-Score ('weighted')
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # 판다스 데이터프레임으로 결과 정리
    performance_metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Value': [accuracy, precision, recall, f1_score]
    })

    # 데이터프레임 출력
    print(f"\n[{i*100+100} epoch result]\n", performance_metrics)

Epoch 1:   0%|          | 0/43 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 43/43 [01:42<00:00,  2.39s/it]


	Loss: 4.6701, Val_Loss: 4.6486, Total Mean Loss: 4.6594, LR: 0.0008000899999999999, Duration: 104.06 sec - model saved!


Epoch 2: 100%|██████████| 43/43 [01:39<00:00,  2.31s/it]


	Loss: 4.6242, Val_Loss: 4.3898, Total Mean Loss: 4.5070, LR: 0.00160008, Duration: 100.48 sec - model saved!


...

Epoch 4: 100%|██████████| 43/43 [01:39<00:00,  2.32s/it]


	Loss: 4.4511, Val_Loss: 3.9385, Total Mean Loss: 4.1948, LR: 0.00320006, Duration: 100.96 sec - model saved!


Epoch 5: 100%|██████████| 43/43 [01:39<00:00,  2.31s/it]


	Loss: 4.4286, Val_Loss: 3.8703, Total Mean Loss: 4.1494, LR: 0.004000050000000001, Duration: 100.65 sec - model saved!


Epoch 6:  81%|████████▏ | 35/43 [01:22<00:18,  2.37s/it]

In [16]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')
model_save = False

for i in range(epochs // 100):
    for epoch in range(100):
        model.train()
        start_time = time.time()
        running_loss = 0.0
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1 + i*100}")
        
        for _, data in pbar:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs, labels = mixup_fn(inputs, labels)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
                
            loss.backward()
            # 그라디언트 클리핑 적용
            clip_grad_norm_(model.parameters(), max_norm=max_norm)
            optimizer.step()
            
            # EMA 모델 업데이트, 필요한 경우
            if model_ema is not None:
                model_ema.update(model)
                
            scheduler.step()
                
            lr = optimizer.param_groups[0]["lr"]
            lrs.append(lr)
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in valid_loader:
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        val_loss /= len(valid_loader)
        val_losses.append(val_loss)
        
        # 모델 저장 조건 수정
        total_loss = val_loss + epoch_loss
        if total_loss < best_loss:
            best_loss = total_loss
            # torch.save(model.state_dict(), model_path)
            model_save = True
            save_text = ' - model saved!'
        else:
            save_text = ''

        epoch_duration = time.time() - start_time
        training_time += epoch_duration
        
        text = f'\tLoss: {epoch_loss:.4f}, Val_Loss: {val_loss:.4f}, Total Mean Loss: {total_loss/2:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec{save_text}'
        print(text)

    # 예측 수행 및 레이블 저장
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 혼동 행렬 생성
    cm = confusion_matrix(all_labels, all_preds)

    # 예측과 실제 레이블
    y_true = all_labels  # 실제 레이블
    y_pred = all_preds  # 모델에 의해 예측된 레이블

    # 전체 데이터셋에 대한 정확도
    accuracy = accuracy_score(y_true, y_pred)

    # 평균 정밀도, 리콜, F1-Score ('weighted')
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    # 판다스 데이터프레임으로 결과 정리
    performance_metrics = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Value': [accuracy, precision, recall, f1_score]
    })

    # 데이터프레임 출력
    print(f"\n[{i*100+100} epoch result]\n", performance_metrics)

Epoch 1: 100%|██████████| 53/53 [01:41<00:00,  1.92s/it]


	Loss: 4.5950, Val_Loss: 4.1453, Total Mean Loss: 4.3702, LR: 0.00080009, Duration: 103.15 sec - model saved!


Epoch 2: 100%|██████████| 53/53 [01:39<00:00,  1.88s/it]


	Loss: 4.4743, Val_Loss: 4.0018, Total Mean Loss: 4.2381, LR: 0.0016000800000000003, Duration: 100.79 sec - model saved!


Epoch 3: 100%|██████████| 53/53 [01:40<00:00,  1.89s/it]


...

	Loss: 2.5952, Val_Loss: 0.8477, Total Mean Loss: 1.7215, LR: 9.843677163216202e-06, Duration: 103.13 sec


Epoch 99: 100%|██████████| 53/53 [01:40<00:00,  1.90s/it]


	Loss: 2.5665, Val_Loss: 0.8481, Total Mean Loss: 1.7073, LR: 2.5366614649679064e-06, Duration: 102.18 sec


Epoch 100: 100%|██████████| 53/53 [01:40<00:00,  1.90s/it]


	Loss: 2.5940, Val_Loss: 0.8485, Total Mean Loss: 1.7212, LR: 1e-07, Duration: 101.85 sec

[100 epoch result]
       Metric     Value
0   Accuracy  0.858000
1  Precision  0.885246
2     Recall  0.858000
3   F1 Score  0.852777
